https://bit.ly/RSML-3-colab

## Коллаборативная фильтрация

### item-to-item

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

In [ ]:
!unzip MovieLens.zip

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [ ]:
movies_with_ratings.head()

In [ ]:
movies_with_ratings.title.value_counts()

In [ ]:
movies_with_ratings.userId.value_counts()

In [ ]:
num_users = movies_with_ratings.userId.unique().shape[0]
num_users

In [ ]:
movies_with_ratings.head()

In [ ]:
movie_vector = {}

for movie, group in tqdm(movies_with_ratings.groupby('title')):
    movie_vector[movie] = np.zeros(num_users)
    
    for i in range(len(group.userId.values)):
        u = group.userId.values[i]
        r = group.rating.values[i]
        movie_vector[movie][int(u - 1)] = r

In [ ]:
movie_vector['Toy Story (1995)']

In [ ]:
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard

In [ ]:
my_fav_film = 'Fight Club (1999)'

titles = []
distances = []

for key in tqdm(movie_vector.keys()):
    if key == my_fav_film:
        continue
    
    titles.append(key)
    distances.append(cosine(movie_vector[my_fav_film], movie_vector[key]))

len(distances)

In [ ]:
distances[:10]

In [ ]:
best_indexes = np.argsort(distances)[:10]
best_indexes

In [ ]:
best_movies = [(titles[i], distances[i]) for i in best_indexes]

for m in best_movies:
    print(m)

### Surprise

https://surpriselib.com/

In [1]:
!pip3 install surprise

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [3]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [7]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [8]:
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [9]:
ratings.rating.min()

0.5

In [10]:
ratings.rating.max()

5.0

In [11]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
# data_1 = Dataset.load_from_df(movies_with_ratings, reader)

In [12]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

In [13]:
dataset['uid'].nunique(), dataset['iid'].nunique()

(610, 9719)

#### user_based

In [14]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': True  # compute  similarities between users
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [15]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8985


0.898482528669153

In [16]:
test_pred[:3]

[Prediction(uid=542, iid='Kill Bill: Vol. 1 (2003)', r_ui=2.5, est=3.7592260572900007, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid=469, iid='Bridge on the River Kwai, The (1957)', r_ui=4.0, est=4.168241951992395, details={'actual_k': 34, 'was_impossible': False}),
 Prediction(uid=317, iid='Scarface (1983)', r_ui=5.0, est=4.062599058241541, details={'actual_k': 49, 'was_impossible': False})]

In [17]:
algo.predict(uid=2, iid='Fight Club (1999)')

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.474085079084162, details={'actual_k': 50, 'was_impossible': False})

#### item_based

In [18]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': False  # compute similarities between items
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [23]:
test_pred = algo.test(testset)

In [24]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8982


0.898181158065249

In [25]:
new_pred = algo.predict(uid=2, iid='Fight Club (1999)')
new_pred

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.2619917274785735, details={'actual_k': 24, 'was_impossible': False})

In [27]:
new_pred

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.2619917274785735, details={'actual_k': 24, 'was_impossible': False})

In [26]:
accuracy.rmse(new_pred, verbose=True)

TypeError: 'int' object is not iterable

In [ ]:
new_pred = algo.predict(uid=20000, iid='Fight Club (1999)')
new_pred

In [ ]:
new_pred = algo.predict(uid=2, iid='unkown')
new_pred

In [ ]:
trainset.global_mean

#### Рекомендации

In [ ]:
def generate_recommendation(uid, model, dataset, thresh=4, amount=5):
    all_titles = list(dataset['iid'].values)
    users_seen_titles = dataset[dataset['uid'] == uid]['iid']
    titles = np.array(list(set(all_titles) - set(users_seen_titles)))

    np.random.shuffle(titles)
    
    rec_list = []
    for title in titles:
        review_prediction = model.predict(uid=uid, iid=title)
        rating = review_prediction.est

        if rating >= thresh:
            rec_list.append((title, round(rating, 2)))
            
            if len(rec_list) >= amount:
                return rec_list

In [ ]:
generate_recommendation(2, algo, dataset)